In [ ]:
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

import os
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

model_name = os.getenv("LLM_MODEL") or "gpt-4o-mini"
model_provider = os.getenv("LLM_MODEL_PROVIDER") or "openai"

current_dir = Path.cwd()
data_dir = current_dir.parent / "data"
index_dir = current_dir.parent / "index"

restaurant_faiss = index_dir / "restaurant-faiss"
restaurant_text = data_dir / "restaurants.txt"


In [2]:
def create_faiss_index():
    loader = TextLoader(str(restaurant_text))
    documents = loader.load()

    text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=50)
    chunks = text_splitter.create_documents(documents)

    embeddings = OpenAIEmbeddings(
        model="text-embedding-3-large",
    )

    db = FAISS.from_documents(chunks, embeddings)
    db.save_local(str(restaurant_faiss))
    
    print("Faiss Index created and saved")


In [3]:
def load_faiss_index():
    embeddings = OpenAIEmbeddings(
        model="text-embedding-3-large",
    )
    load_db = FAISS.load_local(
        str(restaurant_faiss),
        embeddings,
        allow_dangerous_deserialization=True,
    )

    return load_db

In [4]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

In [11]:
def answer_question(db, query):
    llm = ChatOpenAI(model=model_name)
    prompt_template = """
    당신은 유능한 AI 비서입니다. 주어진 맥락 정보를 바탕으로 사용자의 질문에 정확하고 도움이 되는 답변을 제공해야 합니다.
    맥락: {context}
    질문: {question}
    답변을 작성할 때 다음 지침을 따르세요:
    1. 주어진 맥락 정보에 있는 내용만을 사용하여 답변하세요.
    2. 맥락 정보에 없는 내용은 답변에 포함하지 마세요.
    3. 질문과 관련이 없는 정보는 제외하세요
    4. 답변은 간결하고 명확하게 작성하세요.
    5. 불확실한 경우, "주어진 정보로는 정확한 답변을 드릴 수 없습니다."라고 답변하세요.
    답변: 
    """

    prompt = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"],
    )

    qa_chain = (
        {
            "context": db.as_retriever() | format_docs,
            "question": RunnablePassthrough()
        }
        | prompt
        | llm
        | StrOutputParser()
    )

    #result = qa_chain.invoke({"input": query})
    result = qa_chain.invoke(query)
    return result

In [12]:
def main():
    if not os.path.exists(str(restaurant_faiss)):
        create_faiss_index()

    db = load_faiss_index()
    while True:
        query = input("레스토랑에 대해서 궁금한 점을 물어보세요 (종료하려면 'quit' 입력): ")
        if query.lower() == "quit":
            print("프로그램을 종료합니다.")
            break
        answer = answer_question(db, query)
        print(f"답변: {answer}\n")
        
    

In [13]:
main()

답변: 네, 음직점에서는 다양한 아이들을 위한 키즈 메뉴를 제공하고 있습니다.

답변: 음직점에서 유명한 요리는 무스케이크입니다.

답변: 주변에는 무료 주차장이 마련되어 있어 편리하게 이용하실 수 있습니다.

답변: 음직점에서 유명한 요리는 트러플 오일을 사용한 크림 파스타입니다.

답변: 음직점에서는 신선하고 고품질의 식재료를 사용하며, 주로 지역에서 직접 공급받아 신선함을 유지하고 있습니다. 또한 지역에서 생산된 신선한 채소와 고기를 적극 활용합니다.

답변: 음직점에서는 고품질 아라비카 원두를 사용하며, 에스프레소, 라떼, 아메리카노 등 다양한 커피 스타일과 특별한 시그니처 커피 메뉴를 제공합니다.

답변: 네, 음직점에서는 다양한 시그니처 칵테일 프로그램과 칵테일 메뉴를 제공하고 있습니다.

답변: 주어진 정보로는 정확한 답변을 드릴 수 없습니다.

프로그램을 종료합니다.
